In [9]:
import scanpy as sc
import pandas as pd
import numpy as np
import loompy as lp
from scipy import sparse

In [10]:
H5 = "CTR9_snRNASeq/KO/filtered_feature_bc_matrix.h5" 

adata = sc.read_10x_h5(H5)
adata.var_names_make_unique()

# Basic QC filters (tweak as needed)
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Keep raw counts in a layer for later
adata.layers["counts"] = adata.X.copy()

adata

/opt/anaconda3/envs/pyscenic/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/anaconda3/envs/pyscenic/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4888 × 22430
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    layers: 'counts'

In [11]:
loom_out = "CTR9_KO.loom"

X = adata.layers["counts"]
if sparse.issparse(X):
    X = X.T.tocsc()   # loom wants genes x cells
else:
    X = X.T

row_attrs = {"Gene": np.array(adata.var_names)}
col_attrs = {"CellID": np.array(adata.obs_names)}

lp.create(loom_out, X, row_attrs, col_attrs)
loom_out

'CTR9_KO.loom'

In [12]:
LOOM_IN = "CTR9_KO.loom"

# SCENIC resources
TF_LIST = "pyscenic_resources/allTFs_mm.txt"
DBS = [
    "pyscenic_resources/databases/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather",
    "pyscenic_resources/databases/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.scores.feather"
]
MOTIF_ANNOT = "pyscenic_resources/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl"

In [ ]:
## in terminal:

conda activate pyscenic

arboreto_with_multiprocessing.py \
  CTR9_KO.loom \
  pyscenic_resources/allTFs_mm.txt \
  --method grnboost2 \
  --output CTR9_KO.adjacencies.tsv \
  --num_workers 8 \
  --seed 777